In [0]:
%pip install tabula-py pdfplumber pandas

import tabula
import pandas as pd
import os
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

print("🚀 Starting Daily PDF to CSV ETL Pipeline...")

# Step 1: Auto-detect LATEST PDF
pdf_dir = "files/aqi/raw"
pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith('.pdf')]

latest_pdf = None
latest_ctime = -1
for f in pdf_files:
    ctime = os.path.getctime(os.path.join(pdf_dir, f))
    if ctime > latest_ctime:
        latest_pdf = f
        latest_ctime = ctime

pdf_path = os.path.join(pdf_dir, latest_pdf)
print(f"📄 Processing: {latest_pdf}")

# Extract date from filename
yyyymmdd = latest_pdf.split('_')[2].split('.')[0]
report_date = datetime.strptime(yyyymmdd, "%Y%m%d").date()
print(f"📅 Report date: {report_date}")

# Step 2: Extract ALL tables
dfs = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True, pandas_options={'header': None})
print(f"📊 Found {len(dfs)} tables")

# Step 3: Filter AQI tables
relevant_dfs = []
for i, df in enumerate(dfs):
    if df.shape[0] < 3 or df.shape[1] < 6:
        continue
    sample_text = ' '.join(df.iloc[0:3].astype(str).values.flatten()).lower()
    if any(keyword in sample_text for keyword in ['s.no', 'city', 'poor', 'moderate', 'satisfactory']):
        print(f"✅ Table {i+1} is AQI data: {df.shape}")
        relevant_dfs.append(df)

# Step 4: Concatenate & clean
aqi_df = pd.concat(relevant_dfs, ignore_index=True)
print(f"📈 Combined shape: {aqi_df.shape}")

# Step 5: Standardize columns
print("🧹 Cleaning columns...")
aqi_df = aqi_df.iloc[1:].reset_index(drop=True)
aqi_df = aqi_df.dropna(how='all')

column_mapping = {0: 'SNo', 1: 'City', 2: 'Category', 3: 'AQI_Value', 4: 'Dominant_Pollutant', 5: 'Stations'}

if aqi_df.shape[1] == 6:
    aqi_df.columns = [column_mapping.get(i, f'Extra_{i}') for i in range(6)]
elif aqi_df.shape[1] > 6:
    aqi_df = aqi_df.iloc[:, :6]
    aqi_df.columns = [column_mapping.get(i, f'Extra_{i}') for i in range(6)]

# Clean & add metadata
for col in ['SNo', 'City', 'Category', 'Dominant_Pollutant', 'Stations']:
    if col in aqi_df.columns:
        aqi_df[col] = aqi_df[col].astype(str).str.strip()
aqi_df['AQI_Value'] = pd.to_numeric(aqi_df['AQI_Value'], errors='coerce')
aqi_df['report_date'] = str(report_date)
aqi_df['ingestion_timestamp'] = pd.Timestamp.now()

# Filter valid rows
aqi_df = aqi_df[
    (aqi_df['City'].str.len() > 1) &
    (aqi_df['AQI_Value'].notna()) &
    (~aqi_df['City'].str.contains('Page|absent|Note|Health', case=False, na=False)) &
    (~aqi_df['SNo'].str.contains('nan', na=False))
].reset_index(drop=True)

print(f"✅ Cleaned data: {aqi_df.shape} rows")
display(aqi_df.head(10))

# Step 6: Save DAILY CSV (One file per day)
delta_dir = "files/aqi/delta/daily"
daily_csv = f"{delta_dir}/aqi_{yyyymmdd}.csv"

os.makedirs(delta_dir, exist_ok=True)
aqi_df['SNo'] = aqi_df['SNo'].astype(str)
aqi_df['ingestion_timestamp'] = aqi_df['ingestion_timestamp'].astype(str)
aqi_df.to_csv(daily_csv, index=False)

print(f"✅ Saved TODAY'S data: {daily_csv} ({len(aqi_df)} rows)")

# Step 7: Spark view of ALL daily files
all_csvs = [f for f in os.listdir(delta_dir) if f.startswith('aqi_') and f.endswith('.csv')]
print(f"📁 Found {len(all_csvs)} daily CSV files")

if all_csvs:
    combined_df = pd.concat([pd.read_csv(os.path.join(delta_dir, f)) for f in all_csvs], ignore_index=True)
    spark_df = spark.createDataFrame(combined_df)
    spark_df.createOrReplaceTempView("aqi_daily")
    
    print(f"\n🏭 TOP 10 MOST POLLUTED (ALL DAYS):")
    display(spark.sql("SELECT * FROM aqi_daily ORDER BY AQI_Value DESC LIMIT 10"))
    
    print(f"📊 TOTAL records across {len(all_csvs)} days: {spark_df.count()}")

print("\n🎉 DAILY ETL COMPLETE!")
print(f"📂 Folder: files/aqi/delta/daily/")
print(f"📊 Query: SELECT * FROM aqi_daily")
